# Lab Recap

Can we retrieve 3 random facts from the [Internet Chuck Norris Database](http://www.icndb.com/api/) and display them as formatted HTML?

## Cryptographic Hashing

Key terms:
* Hashing: Transforming data of any size into a fixed-size representation (a hash). Typically:
  * If the input data is changed even slightly, the hash is completely different.
  * If you have the hash, you can't determine the exact input that was used, but you may be able to determine a *possible* input.
* Cryptographic Hashing: Hashing where it's computationally difficult to determine a possible input.

Can we write a simple hash function?

Can we create a cryptographic hash for this string?

In [ ]:
important_message = "The exam is on the 12th of May"

## Digital Signing

By combining hashing and asymmetric encryption, we have a way of "signing" digital documents.

Can we digitally sign the following string?

In [ ]:
verified_message = "Don't forget to double-check your answers in an exam"

# Searching

Search algorithms are a well studied class of algorithms as many problems can be expressed as search problems.

As an example, consider this "map" of Romania

<img src="https://d3i71xaburhd42.cloudfront.net/437af7588c6a36fd55c410b7f92b7f47ef57653b/5-Figure3.2-1.png" alt="Romania" />

### Depth-first search

In depth-first search, we explore as far as possible along each path before going back and finding another one.

<img src="https://d18l82el6cdm1i.cloudfront.net/uploads/mf7THWHAbL-mazegif.gif" />
<small>Animation from here: https://brilliant.org/wiki/depth-first-search-dfs/</small>

This is the map of Romania as a dictionary of dictionaries:

In [ ]:
romania = {
    'Arad': { 'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118 },
    'Zerind': { 'Arad': 75, 'Oradea': 71 },
    'Sibiu': { 'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80 },
    'Timisoara': { 'Arad': 118, 'Lugoj': 111 },
    'Oradea': { 'Zerind': 71, 'Sibiu': 151 },
    'Lugoj': { 'Timisoara': 111, 'Mehadia': 70 },
    'Fagaras': { 'Sibiu': 99, 'Bucharest': 211 },
    'Rimnicu Vilcea': { 'Sibiu': 80, 'Pitesti': 97, 'Craiova': 146 },
    'Mehadia': { 'Lugoj': 70, 'Dobreta': 75 },
    'Bucharest': { 'Fagaras': 211, 'Pitesti': 101, 'Urziceni': 85, 'Giurglu': 90 },
    'Pitesti': { 'Rimnicu Vilcea': 97, 'Bucharest': 101, 'Craiova': 138 },
    'Craiova': { 'Rimnicu Vilcea': 146, 'Pitesti': 138, 'Dobreta': 120 },
    'Dobreta': { 'Mehadia': 75, 'Craiova': 120 },
    'Urziceni': { 'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142 },
    'Giurglu': { 'Bucharest': 90 },
    'Hirsova': { 'Urziceni': 98, 'Eforie': 86 },
    'Vaslui': { 'Urziceni': 142, 'Lasi': 92 },
    'Eforie': { 'Hirsova': 86 },
    'Lasi': { 'Vaslui': 92, 'Neamt': 87 },
    'Neamt': { 'Lasi': 87 }
}

Find a path from Arad to Bucharest using depth first search

### Breadth-first search

In breadth-first search, we explore by moving outward from the start along all possible paths.

Find a path from Arad to Bucharest using breadth-first search

#### Lab 4 challenge

Write a function, `nearby(city)`, that returns a list containing all the cities within a 300 mile trip of the the given city, assuming you can only travel along the roads given. 

### Uniform cost search
In uniform-cost search, we explore by moving outward from the start, but exploring along the shortest path found so far first.

Find the shortest path from Arad to Bucharest using uniform-cost search.

# Sorting

Sorting algorithms are one of the most frequently studied class of algorithms. Not only are there many such algorithms, it is also very easy to implement them incorrectly.

Can we implement [Bubble sort](https://en.wikipedia.org/wiki/Bubble_sort) in Python?

See `sorting.py`.

Can we also implement [merge sort](https://en.wikipedia.org/wiki/Merge_sort)?

See `sorting.py`.

# Error Handling

What do we do when things go wrong?

## Using `None`

Consider a function `find(needle, haystack)` that finds the position of `needle` in the list `haystack`. Can we implement this function?

Does python have built-in functionality for doing this?

## Exceptions

Can we change our `find()` function so that it raises an exception?

Consider the function, `score(card_points, cards)`, from the week 4 lab. It takes in a dictionary representing point values for different cards and a list of cards, and calculates the total score for that list.

Can we implement this function differently using exceptions?

A number, `x`, inside a list, `list`, is said to lead back to itself in 1 step if `list[x] == x`. A number leads back to itself in 2 steps if `list[list[x]] == x`. Which of the numbers in the following list lead back to themselves in 3 steps.

In [ ]:
list = [0, 2, 1, 4, 5, 3, 9, 6, 7]

# Alternative ways of working with data structures

Can we implement the `score(card_points, cards)` function without conditionals or exception handling?

## List/Dictionary Comprehensions

Can we extract a list of zids from these email addresses?

In [ ]:
emails = ["z1234567@student.unsw.edu.au", "z7654321@unsw.edu.au", "z7891234@ad.unsw.edu.au", "z1357924@student.unsw.edu.au"]

Can we create a dictionary with the zids as keys and the email addresses as values?

Find all the words that are palindromes in this list:

In [ ]:
words = ["kayak", "hello", "racecar", "madam", "moon", "noon", "shish", "level"]